In [14]:
import re
import asyncio
from aiohttp import TCPConnector, ClientSession
import nest_asyncio
import asyncio
import tracemalloc


import pyecharts.options as opts
from pyecharts.charts import TreeMap
tracemalloc.start()
"""
Gallery 使用 pyecharts 1.1.0
参考地址: https://echarts.apache.org/examples/editor.html?c=treemap-drill-down

目前无法实现的功能:

1、层级的样式配置
"""

nest_asyncio.apply()



async def get_json_data(url: str) -> dict:
    async with ClientSession(connector=TCPConnector(ssl=False)) as session:
        async with session.get(url=url) as response:
            return await response.json()


# 获取官方的数据


loop = asyncio.get_event_loop()
async def main():
    # 在一個異步函數中等待協程完成
    data = await get_json_data(
        url="https://echarts.apache.org/examples/data/asset/data/ec-option-doc-statistics-201604.json"
    )

    tree_map_data = {"children": []}  # 示例，具體依賴您的邏輯

        



    def convert(source, target, base_path: str):
        for key in source:
            if base_path != "":
                path = base_path + "." + key
            else:
                path = key
            if re.match(r"/^\$/", key):
                pass
            else:
                child = {"name": path, "children": []}
                target["children"].append(child)
                if isinstance(source[key], dict):
                    convert(source[key], child, path)
                else:
                    target["value"] = source["$count"]


    convert(source=data, target=tree_map_data, base_path="")

    treeMap = TreeMap(init_opts=opts.InitOpts(width="1200px", height="720px"))
    treeMap = treeMap.add(
        series_name="option",
        data=tree_map_data["children"],
        visual_min=300,
        leaf_depth=1,
        label_opts=opts.LabelOpts(position="inside")
    )
    treeMap = treeMap.set_global_opts(
        legend_opts=opts.LegendOpts(is_show=False),
        title_opts=opts.TitleOpts(
            title="Echarts 配置项查询分布", subtitle="2016/04", pos_left="leafDepth"
        )
    )
    treeMap.render("echarts_option_query.html")

asyncio.run(main())
tracemalloc.stop()

In [ ]:
import re
import asyncio
from aiohttp import TCPConnector, ClientSession
import nest_asyncio


import pyecharts.options as opts
from pyecharts.charts import TreeMap

"""
Gallery 使用 pyecharts 1.1.0
参考地址: https://echarts.apache.org/examples/editor.html?c=treemap-drill-down

目前无法实现的功能:

1、层级的样式配置
"""

nest_asyncio.apply()

async def get_json_data(url: str) -> dict:
    async with ClientSession(connector=TCPConnector(ssl=False)) as session:
        async with session.get(url=url) as response:
            return await response.json()


# 获取官方的数据
loop = asyncio.get_event_loop()
data = loop.run_until_complete(
    get_json_data(
        url="https://echarts.apache.org/examples/data/asset/data/ec-option-doc-statistics-201604.json"
    )
)

tree_map_data: dict = {"children": []}


def convert(source, target, base_path: str):
    for key in source:
        if base_path != "":
            path = base_path + "." + key
        else:
            path = key
        if re.match(r"/^\$/", key):
            pass
        else:
            child = {"name": path, "children": []}
            target["children"].append(child)
            if isinstance(source[key], dict):
                convert(source[key], child, path)
            else:
                target["value"] = source["$count"]


convert(source=data, target=tree_map_data, base_path="")


(
    TreeMap(init_opts=opts.InitOpts(width="1200px", height="720px"))
    .add(
        series_name="option",
        data=tree_map_data["children"],
        visual_min=300,
        leaf_depth=1,
        # 标签居中为 position = "inside"
        label_opts=opts.LabelOpts(position="inside"),
    )
    .set_global_opts(
        legend_opts=opts.LegendOpts(is_show=False),
        title_opts=opts.TitleOpts(
            title="Echarts 配置项查询分布", subtitle="2016/04", pos_left="leafDepth"
        ),
    )
    .render("echarts_option_query.html")
)